In [40]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
from variable_names import to_keep
import missingno as msno
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# eye candy plots
plt.style.use('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-dark.mplstyle')
# source https://github.com/dhaitz/matplotlib-stylesheets


In [41]:
### Here we make some arguments about why we drop certain variables

## Fill in later


In [42]:
df = pd.read_csv('data/Train.csv')
df = df[to_keep]

In [43]:
df.head()

,Date,Place_ID,target,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_tropospheric_NO2_column_number_density,...,L3_HCHO_tropospheric_HCHO_column_number_density,L3_HCHO_tropospheric_HCHO_column_number_density_amf,L3_CLOUD_cloud_top_pressure,L3_CLOUD_surface_albedo,L3_CLOUD_cloud_fraction,L3_CLOUD_cloud_optical_depth,L3_AER_AI_absorbing_aerosol_index,L3_SO2_SO2_column_number_density,L3_SO2_SO2_column_number_density_amf,L3_SO2_absorbing_aerosol_index
0,2020-01-02,010Q650,38.0,11.000000,60.200001,0.00804,18.516840,1.996377,-1.227395,0.000017,...,0.000064,0.566828,NaN,NaN,0.000000,NaN,-1.231330,-0.000127,0.312521,-1.861476
1,2020-01-03,010Q650,39.0,14.600000,48.799999,0.00839,22.546533,3.330430,-1.188108,0.000021,...,0.000171,0.858446,99353.672374,0.315403,0.059358,5.958538,-1.082553,0.000150,0.433957,-1.452612
2,2020-01-04,010Q650,24.0,16.400000,33.400002,0.00750,27.031030,5.065727,3.500559,0.000007,...,0.000124,0.910536,95671.383578,0.307463,0.082247,5.755760,-1.001241,0.000150,0.356925,-1.572950
3,2020-01-05,010Q650,49.0,6.911948,21.300001,0.00391,23.971857,3.004001,1.099468,0.000023,...,0.000081,1.132571,96232.486418,0.279637,0.121555,6.246885,-0.777023,0.000227,0.584522,-1.239317
4,2020-01-06,010Q650,21.0,13.900001,44.700001,0.00535,16.816309,2.621787,2.670559,0.000009,...,0.000140,0.649359,52561.523079,0.238241,0.037008,4.205691,0.366324,0.000390,0.408047,0.202489


## Test train split for time-series data

In [44]:


def test_train_time_split(data,target_name,date_name,test_size=0.25):
    '''Returns train-test split for time series data, ensuring that 
    data later in time are preserved for the test sample'''

    sorted_data = data.sort_values(date_name)
    n_obs = data.shape[0]
    arg_split = int( n_obs * (1. - test_size) )

    train, test = sorted_data[0:arg_split+1], sorted_data[arg_split+1:]
    X_train, X_test = train.drop(target_name,axis=1), test.drop(target_name, axis=1)
    y_train, y_test = train[target_name], test[target_name]

    return X_train, X_test, y_train, y_test


In [45]:
#Perform test train split for time series data

X_train, X_test, y_train, y_test = test_train_time_split(data=df,
                                                         target_name='target',
                                                         date_name='Date',
                                                         test_size=0.3)

## Data Preprocessing Pipepline, without standardisation

In [46]:
qnt_cols = list(X_train.columns.drop(['Date', 'Place_ID']))
cat_cols = ['Place_ID']


qnt_pipeline = Pipeline([
    ('impute_qnt', SimpleImputer(strategy='mean'))
])

cat_pipeline = Pipeline([
    ('impute_cat', SimpleImputer(strategy='constant', fill_value='missing')),
    ('ohe_cat', OneHotEncoder(handle_unknown='ignore'))
])


preprocessor = ColumnTransformer([
    ('qnt', qnt_pipeline, qnt_cols),
    ('cat', cat_pipeline, cat_cols)
])



In [47]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [48]:
## Later I want to do a fillna with local mean based on Place_ID